http://jeroenjanssens.com/2013/09/19/seven-command-line-tools-for-data-science.html

In [15]:
import numpy as np
from os import path
data_dir = "/home/daniel/git/Python2.7/DataScience/nlp_training_data"

chap_line_nums = np.loadtxt(path.join(data_dir, 'alice_chap_nums.txt'), dtype='int', delimiter='\n')

f = open(path.join(data_dir,'alice.txt'), 'r')
lines = f.readlines()
f.close()
     
print chap_line_nums

[ 212  422  632  891 1160 1461 1792 2137 2454 2778 3077 3343 3637]


In [62]:
for i in range(chap_line_nums.size - 1):
    chap_num = i + 1
    f = open(path.join(data_dir,'chap_%02i.txt' %chap_num), 'w')
    
    start = chap_line_nums[i]
    end = chap_line_nums[i+1] - 1

    for line in lines[start:end]:
        f.write(line) 
    
    f.close()
    

In [28]:
from nltk import tokenize

In [29]:
def get_chap_sents(filename):
    f = open(filename, 'r')
    lines = f.read()
    f.close()

    sents = tokenize.sent_tokenize(lines)
    sents = map(lambda s:s.replace('\r\n',' '), sents)
    
    return sents

In [68]:
import glob
import pandas as pd

chps = glob.glob(path.join(data_dir,"chap_*.txt"))
chps.sort()

sentences = get_chap_sents(chps[0])
df = pd.DataFrame()

df['lines'] = sentences
df['chap'] = np.ones(len(sentences))

In [74]:
for ind, _file in enumerate(chps[1:], start=2):
    sentences = get_chap_sents(_file)
    n = len(sentences)
    temp = pd.DataFrame({ 'lines':sentences, 'chap':np.ones(n)*ind })
    #print temp
    df = pd.concat([df, temp], axis=0)
    

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS

In [95]:
stop_words = ENGLISH_STOP_WORDS.union('alice')

In [96]:
vectorizer = TfidfVectorizer(max_features=100, # recommended on sklearn pages
                             max_df=0.65,
                             min_df=0.01,
                             use_idf=True,
                             stop_words=stop_words,
                             norm='l2',
                             ngram_range=(1,1),
                             analyzer='word',
                             decode_error='ignore',
                             strip_accents='unicode'
                             )


matrix = vectorizer.fit_transform(df['lines'])
features = vectorizer.get_feature_names()
       
print matrix.shape 
print features
# vectorize the "docs"
# kmeans and store cluster in df

(11227, 99)
[u'added', u'alice', u'asked', u'away', u'began', u'came', u'cat', u'caterpillar', u'come', u'course', u'court', u'cried', u'curious', u'day', u'dear', u'did', u'doesn', u'don', u'door', u'dormouse', u'duchess', u'end', u'eyes', u'feet', u'felt', u'getting', u'going', u'good', u'got', u'great', u'gryphon', u'half', u'hand', u'hare', u'hatter', u'head', u'heard', u'house', u'illustration', u'jury', u'just', u'king', u'know', u'large', u'left', u'let', u'like', u'little', u'll', u'long', u'look', u'looked', u'looking', u'make', u'march', u'minute', u'mock', u'moment', u'mouse', u'oh', u'old', u'poor', u'queen', u'question', u'quite', u'rabbit', u'replied', u'right', u'round', u'said', u'say', u'shall', u'soon', u'sort', u'spoke', u'sure', u'talking', u'tea', u'tell', u'thing', u'things', u'think', u'thought', u'till', u'time', u'tone', u'took', u'turned', u'turtle', u'use', u've', u'voice', u'way', u'went', u'white', u'wish', u'won', u'wonder', u'words']


In [85]:
from sklearn.cluster import KMeans
k = len(chps)
km = KMeans(k)
assignments = km.fit_predict(matrix)

    


In [86]:
df['cluster_chaps'] = assignments

In [87]:
df


,chap,lines,cluster_chaps
0,1,[Sidenote: _Down the Rabbit-Hole_] ALICE wa...,5
1,1,So she was considering in her own mind (as wel...,2
2,1,There was nothing so _very_ remarkable in that...,5
3,1,Oh dear!,9
4,1,"I shall be too late!""",9
5,1,"(when she thought it over afterwards, it occur...",5
6,1,"In another moment down went Alice after it, ne...",5
7,1,The rabbit-hole went straight on like a tunnel...,5
8,1,"[Illustration] Either the well was very deep,...",5
9,1,"First, she tried to look down and make out wha...",5
